In [1]:
# ============================================================
# CONFIGURACIÓN E IMPORTS
# ============================================================

import re
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from pathlib import Path
import sys
import os

# Agregar la raíz del proyecto al path
ROOT = Path().resolve().parent
sys.path.insert(0, str(ROOT))

# Importar configuración centralizada
from config import PATHS, CONFIG

In [ ]:
# ============= CONFIGURACIÓN USANDO CONFIG.PY =============

# Usar rutas centralizadas
CARPETA_DATOS_CRUDOS = PATHS.RAW_FB
CARPETA_PROCESADOS = PATHS.UNCLEANED

# Usar configuración centralizada
PREFIJO_ID = CONFIG.PREFIJO_ID
NUMERO_INICIAL = CONFIG.NUMERO_INICIAL
USAR_FILTRO_FECHA = CONFIG.USAR_FILTRO_FECHA
MESES_ATRAS = CONFIG.MESES_ATRAS
FECHA_DESDE = CONFIG.FECHA_DESDE

print("="*60)
print("CONFIGURACIÓN ACTIVA:")
print("="*60)
print(f"Datos crudos: {CARPETA_DATOS_CRUDOS}")
print(f"Procesados: {CARPETA_PROCESADOS}")
print(f"Prefijo ID: {PREFIJO_ID}")
print(f"Filtro de fecha: {'Activo' if USAR_FILTRO_FECHA else 'Inactivo'}")
if USAR_FILTRO_FECHA:
    if FECHA_DESDE:
        print(f"  Desde: {FECHA_DESDE}")
    else:
        print(f"  Últimos {MESES_ATRAS} meses")
print("="*60)

In [3]:
def normalizar_fecha(fecha_str):
    """Normaliza diferentes formatos de fecha a YYYY-MM-DD HH:MM:SS"""
    if pd.isna(fecha_str) or fecha_str == "":
        return ""
    
    if isinstance(fecha_str, datetime):
        return fecha_str.strftime('%Y-%m-%d %H:%M:%S')
    
    fecha_str = str(fecha_str).strip()
    
    try:
        # Formato ISO con timezone (2023-01-16T11:34:28Z)
        if 'T' in fecha_str:
            fecha_str = fecha_str.replace('Z', '')
            dt = datetime.fromisoformat(fecha_str.split('+')[0].split('-0')[0])
            return dt.strftime('%Y-%m-%d %H:%M:%S')
        
        # Formato WhatsApp español: "27/2/25 1:42 p. m."
        if 'p. m.' in fecha_str or 'a. m.' in fecha_str:
            fecha_str_limpia = fecha_str.replace('p. m.', 'PM').replace('a. m.', 'AM')
            partes = fecha_str_limpia.split()
            
            if len(partes) >= 2:
                fecha_nums = partes[0].split('/')
                dia, mes, anio = int(fecha_nums[0]), int(fecha_nums[1]), int(fecha_nums[2])
                
                if anio < 100:
                    anio = 2000 + anio if anio < 50 else 1900 + anio
                
                hora_nums = partes[1].split(':')
                hora = int(hora_nums[0])
                minuto = int(hora_nums[1]) if len(hora_nums) > 1 else 0
                am_pm = partes[2] if len(partes) > 2 else "AM"
                
                if am_pm == 'PM' and hora != 12:
                    hora += 12
                elif am_pm == 'AM' and hora == 12:
                    hora = 0
                
                dt = datetime(anio, mes, dia, hora, minuto, 0)
                return dt.strftime('%Y-%m-%d %H:%M:%S')
        
        # Formato M/D/YYYY (7/17/2025)
        if '/' in fecha_str and len(fecha_str.split('/')) == 3:
            parts = fecha_str.split('/')
            if len(parts[2]) == 4:
                dt = datetime.strptime(fecha_str, '%m/%d/%Y')
                return dt.strftime('%Y-%m-%d %H:%M:%S')
        
        # Meses en español
        meses_es = {
            'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr',
            'may': 'May', 'jun': 'Jun', 'jul': 'Jul', 'ago': 'Aug',
            'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'
        }
        
        fecha_lower = fecha_str.lower()
        for mes_es, mes_en in meses_es.items():
            if mes_es in fecha_lower:
                fecha_str = fecha_str.lower().replace(mes_es, mes_en)
                break
        
        # Formatos comunes
        formatos = [
            '%b %d, %Y %I:%M:%S %p',
            '%B %d, %Y %I:%M:%S %p',
            '%Y-%m-%d %H:%M:%S',
            '%d/%m/%Y %H:%M:%S',
            '%m/%d/%Y %H:%M:%S',
            '%Y-%m-%d',
            '%d/%m/%Y',
            '%m/%d/%Y',
        ]
        
        for formato in formatos:
            try:
                dt = datetime.strptime(fecha_str, formato)
                return dt.strftime('%Y-%m-%d %H:%M:%S')
            except:
                continue
        
        return fecha_str
        
    except Exception as e:
        return fecha_str
    
def aplicar_filtro_fecha(fecha_str, fecha_limite):
    """Verifica si una fecha cumple con el filtro establecido"""
    if not fecha_limite or not fecha_str:
        return True
    
    try:
        fecha_dt = datetime.strptime(fecha_str, '%Y-%m-%d %H:%M:%S')
        return fecha_dt >= fecha_limite
    except:
        return True  # Si hay error, incluir el registro

In [ ]:
def extraer_html(archivo_html, id_participante, fecha_limite=None):
    """Extrae publicaciones de archivos HTML (Facebook, Instagram, etc.)"""
    
    try:
        with open(archivo_html, 'r', encoding='utf-8') as file:
            html_content = file.read()
    except Exception as e:
        print(f"Error leyendo archivo: {e}")
        return []
    
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = soup.find_all('section', class_='_a6-g')
    posts_data = []
    
    for idx, section in enumerate(sections, 1):
        # Extraer fecha
        fecha_publicacion = None
        footer = section.find('footer')
        time_elem = footer.find('time') if footer else None
        
        if time_elem:
            fecha_publicacion = time_elem.get('datetime')
        else:
            date_div = section.find('div', string=re.compile(r'Actualizado'))
            if date_div:
                fecha_publicacion = date_div.text.replace('Actualizado ', '')
            elif footer:
                date_div = footer.find('div', class_='_a72d')
                if date_div:
                    fecha_publicacion = date_div.text.strip()
        
        # Normalizar fecha
        fecha_normalizada = normalizar_fecha(fecha_publicacion)
        
        # Aplicar filtro de fecha
        if not aplicar_filtro_fecha(fecha_normalizada, fecha_limite):
            continue
        
        # Extraer texto
        texto_publicacion = ""
        content_div = section.find('div', class_='_2ph_')
        
        if content_div:
            text_divs = content_div.find_all('div', recursive=True)
            for div in text_divs:
                text = div.string
                if text and text.strip() and not text.startswith('Actualizado'):
                    if len(text.strip()) > 10:
                        texto_publicacion = text.strip()
                        break
            
            if not texto_publicacion:
                link_elem = content_div.find('a')
                if link_elem and link_elem.get('href'):
                    texto_publicacion = link_elem.get('href')
        
        # Extraer tipo de publicación
        h2 = section.find('h2')
        tipo_publicacion = h2.text.strip() if h2 else ""
        
        posts_data.append({
            'id_participante': id_participante,
            'id_publicacion': f"{id_participante}_POST_{idx:03d}",
            'fuente': 'Facebook',
            'fecha_publicacion': fecha_normalizada,
            'texto_publicacion': texto_publicacion if texto_publicacion else tipo_publicacion
        })
    
    return posts_data

In [ ]:
def extraer_csv_excel(archivo, id_participante, fecha_limite=None):
    """Extrae publicaciones de archivos CSV o Excel"""
    
    try:
        # Detectar tipo de archivo
        if archivo.endswith(('.xlsx', '.xls')):
            df = pd.read_excel(archivo, engine='openpyxl')
        else:
            # Intentar diferentes encodings
            for encoding in ['utf-8', 'latin-1', 'cp1252']:
                try:
                    df = pd.read_csv(archivo, encoding=encoding)
                    break
                except:
                    continue
    except Exception as e:
        print(f"Error leyendo archivo: {e}")
        return []
    
    # Limpiar DataFrame
    df = df.dropna(how='all')
    
    # Detectar columnas
    col_fecha = None
    col_texto = None
    
    for col in df.columns:
        col_lower = col.lower().strip()
        if 'fecha' in col_lower or 'date' in col_lower:
            col_fecha = col
        if any(k in col_lower for k in ['publicacion', 'post', 'texto', 'text', 'content']):
            col_texto = col
    
    if not col_fecha or not col_texto:
        print(f"  ⚠️  Columnas no identificadas. Disponibles: {list(df.columns)}")
        return []
    
    posts_data = []
    
    for idx, row in df.iterrows():
        fecha_publicacion = row[col_fecha] if pd.notna(row[col_fecha]) else ""
        texto_publicacion = row[col_texto] if pd.notna(row[col_texto]) else ""
        
        # Normalizar fecha
        fecha_normalizada = normalizar_fecha(fecha_publicacion)
        
        # Aplicar filtro de fecha
        if not aplicar_filtro_fecha(fecha_normalizada, fecha_limite):
            continue
        
        if fecha_publicacion or texto_publicacion:
            posts_data.append({
                'id_participante': id_participante,
                'id_publicacion': f"{id_participante}_{idx+1:03d}",
                'fuente': 'Facebook',
                'fecha_publicacion': fecha_normalizada,
                'texto_publicacion': str(texto_publicacion)
            })
    
    return posts_data


In [6]:
def procesar_archivo(archivo, id_participante, fecha_limite=None):
    """Procesa un archivo y retorna las publicaciones extraídas"""
    
    archivo_lower = archivo.lower()
    
    # Detectar tipo de archivo
    if archivo_lower.endswith(('.html', '.htm')):
        return extraer_html(archivo, id_participante, fecha_limite)
    elif archivo_lower.endswith(('.xlsx', '.xls', '.csv')):
        return extraer_csv_excel(archivo, id_participante, fecha_limite)
    else:
        # Intentar detectar por contenido
        try:
            with open(archivo, 'r', encoding='utf-8') as f:
                contenido = f.read(1000)
                if '<html' in contenido.lower() or '<!doctype' in contenido.lower():
                    return extraer_html(archivo, id_participante, fecha_limite)
        except:
            pass
        
        # Intentar como CSV
        return extraer_csv_excel(archivo, id_participante, fecha_limite)


In [ ]:
def procesar_carpeta_completa():
    """Procesa todos los archivos en la carpeta de datos crudos"""
    
    print("=" * 80)
    print("EXTRACTOR UNIVERSAL DE PUBLICACIONES")
    print("=" * 80)
    
    # Crear carpeta de procesados si no existe
    if not os.path.exists(CARPETA_PROCESADOS):
        os.makedirs(CARPETA_PROCESADOS)
        print(f"✓ Carpeta '{CARPETA_PROCESADOS}/' creada")
    
    # Calcular fecha límite
    fecha_limite = None
    if USAR_FILTRO_FECHA:
        if FECHA_DESDE:
            fecha_limite = datetime.strptime(FECHA_DESDE, '%Y-%m-%d')
            print(f"\nFiltro: Mensajes desde {FECHA_DESDE}")
        else:
            fecha_limite = datetime.now() - timedelta(days=MESES_ATRAS * 30)
            print(f"\nFiltro: Últimos {MESES_ATRAS} meses (desde {fecha_limite.strftime('%Y-%m-%d')})")
    else:
        print("\nSin filtro de fecha")
    
    # Verificar carpeta
    if not os.path.exists(CARPETA_DATOS_CRUDOS):
        print(f"\nError: La carpeta '{CARPETA_DATOS_CRUDOS}/' no existe")
        return None
    
    # Filtrar archivos: excluir archivos del sistema y ocultos
    archivos_ignorar = {'.ds_store', 'thumbs.db', 'desktop.ini', '.gitkeep', '.gitignore'}
    extensiones_validas = {'.html', '.htm', '.csv', '.xlsx', '.xls'}
    
    archivos = sorted([
        f for f in os.listdir(CARPETA_DATOS_CRUDOS) 
        if os.path.isfile(os.path.join(CARPETA_DATOS_CRUDOS, f))
        and not f.startswith('.')  # Ignorar archivos ocultos
        and f.lower() not in archivos_ignorar  # Ignorar archivos del sistema
        and any(f.lower().endswith(ext) for ext in extensiones_validas)  # Solo extensiones válidas
    ])
    
    if not archivos:
        print(f"\n⚠️  No se encontraron archivos válidos en '{CARPETA_DATOS_CRUDOS}/'")
        print(f"    Extensiones soportadas: {', '.join(extensiones_validas)}")
        return None
    
    print(f"\n📁 Archivos válidos encontrados: {len(archivos)}")
    print("-" * 80)
    
    archivos_guardados = []
    contador = 0
    estadisticas = []
    
    for nombre in archivos:
        ruta = os.path.join(CARPETA_DATOS_CRUDOS, nombre)
        contador += 1
        
        # Extraer número del nombre del archivo
        nombre_sin_ext = os.path.splitext(nombre)[0]
        # Buscar números en el nombre del archivo
        numeros = re.findall(r'\d+', nombre_sin_ext)
        
        if numeros:
            # Usar el primer número encontrado
            numero = int(numeros[0])
            id_participante = f"{PREFIJO_ID}{numero:03d}"
        else:
            # Si no hay número, usar contador como fallback
            id_participante = f"{PREFIJO_ID}{NUMERO_INICIAL + contador - 1:03d}"
            print(f"    ⚠️  No se encontró número en '{nombre}', usando contador")
        
        print(f"\n[{contador}/{len(archivos)}] Procesando: {nombre}")
        print(f"    ID Participante: {id_participante}")
        
        try:
            posts = procesar_archivo(ruta, id_participante, fecha_limite)
            
            if posts:
                # Crear DataFrame para este participante
                df_participante = pd.DataFrame(posts)
                
                # Generar nombre de archivo individual
                nombre_archivo = f"{id_participante}.csv"
                ruta_salida = os.path.join(CARPETA_PROCESADOS, nombre_archivo)

                # Si existe un archivo previo, cargarlo y hacer merge seguro
                if os.path.exists(ruta_salida):
                    try:
                        df_existente = pd.read_csv(ruta_salida, encoding='utf-8-sig')
                        # Asegurar columna 'fuente' en existentes
                        if 'fuente' not in df_existente.columns:
                            df_existente['fuente'] = 'Unknown'
                        inicio_idx = len(df_existente)
                    except Exception as e:
                        print(f"    ⚠️  Error leyendo archivo existente: {e}. Se sobrescribirá si es necesario.")
                        df_existente = pd.DataFrame()
                        inicio_idx = 0
                else:
                    df_existente = pd.DataFrame()
                    inicio_idx = 0

                # Preparar claves para evitar duplicados (fecha + primeros 150 chars del texto)
                if not df_existente.empty:
                    df_existente['clave'] = df_existente['fecha_publicacion'].astype(str) + '_' + df_existente['texto_publicacion'].astype(str).str[:150]
                
                df_participante['fuente'] = 'Facebook'
                df_participante['clave'] = df_participante['fecha_publicacion'].astype(str) + '_' + df_participante['texto_publicacion'].astype(str).str[:150]

                # Filtrar publicaciones que ya existan en el archivo previo
                if not df_existente.empty:
                    df_participante = df_participante[~df_participante['clave'].isin(df_existente['clave'])]

                # Si no hay nuevas publicaciones después del filtrado
                if df_participante.empty:
                    print(f"    ℹ️  No hay publicaciones nuevas para {id_participante}")
                else:
                    # Renumerar id_publicacion en los nuevos a partir de inicio_idx
                    for i, row_idx in enumerate(df_participante.index):
                        df_participante.at[row_idx, 'id_publicacion'] = f"{id_participante}_{inicio_idx + i + 1:03d}"

                    # Concatenar y guardar
                    if not df_existente.empty:
                        df_final = pd.concat([df_existente.drop(columns=['clave'], errors='ignore'), df_participante.drop(columns=['clave'], errors='ignore')], ignore_index=True)
                    else:
                        df_final = df_participante.drop(columns=['clave'], errors='ignore')

                    # Ordenar por id_participante (numérico si es posible) antes de guardar
                    if 'id_participante' in df_final.columns:
                        try:
                            nums = df_final['id_participante'].astype(str).str.extract(r'(\d+)$')[0]
                            if nums.notna().any():
                                df_final['_orden'] = nums.astype(float).astype('Int64')
                                df_final = df_final.sort_values(['_orden', 'id_participante']).drop(columns=['_orden'])
                            else:
                                df_final = df_final.sort_values('id_participante')
                        except Exception:
                            df_final = df_final.sort_values('id_participante')

                    df_final.to_csv(ruta_salida, index=False, encoding='utf-8-sig')

                    print(f"    ✓ {len(df_participante)} publicaciones nuevas extraídas")
                    print(f"    ✓ Guardado como: {nombre_archivo} ({len(df_final)} total)")

                    archivos_guardados.append(nombre_archivo)
                    estadisticas.append({
                        'id_participante': id_participante,
                        'archivo_original': nombre,
                        'total_publicaciones': len(df_participante),
                        'archivo_procesado': nombre_archivo
                    })
            else:
                print(f"    Sin publicaciones")
                
        except Exception as e:
            print(f"    Error: {e}")
            continue
    
    if not archivos_guardados:
        print("\nNo se extrajeron publicaciones de ningún archivo")
        return None
    
    # Crear DataFrame con estadísticas
    df_estadisticas = pd.DataFrame(estadisticas)
    
    print("\n" + "=" * 80)
    print("✓ PROCESO COMPLETADO")
    print("=" * 80)
    print(f"Total de archivos procesados: {len(archivos_guardados)}")
    print(f"Total de publicaciones extraídas: {df_estadisticas['total_publicaciones'].sum()}")
    print(f"Ubicación: {CARPETA_PROCESADOS}/")
    
    # Resumen por participante
    print("\n" + "-" * 80)
    print("RESUMEN POR PARTICIPANTE:")
    print("-" * 80)
    print(df_estadisticas[['id_participante', 'total_publicaciones', 'archivo_procesado']].to_string(index=False))
    
    # Lista de archivos generados
    print("\n" + "-" * 80)
    print("ARCHIVOS GENERADOS:")
    print("-" * 80)
    for archivo in archivos_guardados:
        print(f"  📄 {archivo}")
    
    return df_estadisticas


In [ ]:
if __name__ == "__main__":
    df_resultado = procesar_carpeta_completa()
